Segmenting and clustering neighbourhoods in Toronto

First submission

ran 'conda install -c anaconda beautifulsoup4' from terminal

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Use sessions to scrape wiki page and beautiful soup to parse:

In [2]:
wikisess = requests.session()
wikihtml = wikisess.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
wikisoup = BeautifulSoup(wikihtml.text, 'html.parser')

loop through rows and columns to build list of lists and convert to dataframe:

In [3]:
nhoodrowsv01 = []
for tr in wikisoup.find_all('table')[0].find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.rstrip())
    if len(row) == 3 and row[1] != 'Not assigned':
        if row[2] == 'Not assigned': row[2] = row[1] # if neighbourhood has no name, use borough name
        nhoodrowsv01.append(row)
nhooddfv01 = pd.DataFrame(nhoodrowsv01, columns=['postalcode','borough','neighbourhood'])
nhooddfv01.shape

(103, 3)

if postal codes appear more than once, join up the neighbourhood names:

In [4]:
nhoodrowsv02 = []
for pc in nhooddfv01['postalcode'].unique():
    row = []
    nhoods = 0
    hoodcat = None
    lastborough = None
    for rowind in nhooddfv01.index.values:
        if nhooddfv01.loc[rowind,'postalcode'] == pc:
            if nhoods == 0: hoodcat = nhooddfv01.loc[rowind,'neighbourhood']
            else: hoodcat = hoodcat + ', ' + nhooddfv01.loc[rowind,'neighbourhood']
            lastborough = nhooddfv01.loc[rowind,'borough']
            nhoods = nhoods + 1
    nhoodrowsv02.append([pc, lastborough, hoodcat])
nhooddfv02 = pd.DataFrame(nhoodrowsv02, columns=['postalcode','borough','neighbourhood'])
nhooddfv02.shape

(103, 3)

Second submission

In [5]:
import geocoder

ran variations of the code provided, but it did not work for me:

postal_code = 'M1H 2V6'

lat_lng_coords = None

loop until you get the coordinates
while(lat_lng_coords is None):
g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
lat_lng_coords

Use the csv file provided:

In [6]:
coordsdf = pd.read_csv('https://cocl.us/Geospatial_data')
coordsdf.rename(columns={'Postal Code':'postalcode', 'Latitude':'lat', 'Longitude':'long'}, inplace=True)
coordsdf.head()

,postalcode,lat,long
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join co-ordinates to dataframe from wiki

In [7]:
nhooddfv03 = pd.merge(nhooddfv02, coordsdf, on='postalcode', how='left')
nhooddfv03.head()

,postalcode,borough,neighbourhood,lat,long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Third submission

Based on the code from the New York notebook:

In [8]:
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

In [9]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [10]:
import json
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [11]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="agent001")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('co-ordinates:',latitude, longitude)

co-ordinates: 43.6534817 -79.3839347


Use only boroughs containing 'Toronto':

In [12]:
nhooddfv04 = nhooddfv03[nhooddfv03['borough'].str.contains('Toronto')].reset_index(drop=True)
nhooddfv04.shape

(39, 5)

In [13]:
torontomap = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, borough, neighbourhood in zip(nhooddfv04['lat'], nhooddfv04['long'], nhooddfv04['borough'], nhooddfv04['neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap)  

torontomap

In [14]:
clientid = 'ONYHJ5L0R3SMLDUS12XEFK1KQDWHAQBTQT3W3ZP2BQ3IO42M'
clientsecret = 'AH4YMBNXXU0PU3NYCYEV5VXE52BPO1MBT1WUYXX4EA3AG13W'
version = '20180605'

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        if name == 'Harbourfront East, Union Station, Toronto Islands': continue # no results here
        if name == 'Islington Avenue, Humber Valley Village': continue # no results here
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            clientid, 
            clientsecret, 
            version, 
            lat, 
            lng, 
            radius, 
            nvenues)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
nvenues=100

torontovenues = getNearbyVenues(names=nhooddfv04['neighbourhood'],
                                latitudes=nhooddfv04['lat'],
                                longitudes=nhooddfv04['long']
                                )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetow

In [20]:
print(torontovenues.shape)
torontovenues['Neighborhood'].value_counts()

(1536, 7)


Toronto Dominion Centre, Design Exchange                                                                      100
Commerce Court, Victoria Hotel                                                                                100
Garden District, Ryerson                                                                                      100
First Canadian Place, Underground city                                                                        100
Stn A PO Boxes                                                                                                 97
Richmond, Adelaide, King                                                                                       96
St. James Town                                                                                                 84
Church and Wellesley                                                                                           77
Central Bay Street                                                                      

Clustering does not work well for neighbourhoods with very few venues. Filtering out up to 20:

In [21]:
tvfiltered = torontovenues.groupby('Neighborhood').filter(lambda x: len(x) > 20)
tvfiltered['Neighborhood'].value_counts()

First Canadian Place, Underground city          100
Garden District, Ryerson                        100
Toronto Dominion Centre, Design Exchange        100
Commerce Court, Victoria Hotel                  100
Stn A PO Boxes                                   97
Richmond, Adelaide, King                         96
St. James Town                                   84
Church and Wellesley                             77
Central Bay Street                               67
Kensington Market, Chinatown, Grange Park        63
Berczy Park                                      58
Regent Park, Harbourfront                        46
Little Portugal, Trinity                         44
The Danforth West, Riverdale                     42
St. James Town, Cabbagetown                      41
Studio District                                  40
Davisville                                       37
Runnymede, Swansea                               35
University of Toronto, Harbord                   34
Queen's Park

In [22]:
tvcategoriesv01 = pd.get_dummies(tvfiltered[['Venue Category']], prefix="", prefix_sep="")
tvcategoriesv01['nhoodname'] = tvfiltered['Neighborhood'] #Neighborhood is a type of Foursquare venue..
fixed_columns = [tvcategoriesv01.columns[-1]] + list(tvcategoriesv01.columns[:-1])
tvcategoriesv01 = tvcategoriesv01[fixed_columns]
print(tvcategoriesv01.shape)

(1413, 206)


In [23]:
tvcategoriesv02 = tvcategoriesv01.groupby('nhoodname').mean().reset_index()
tvcategoriesv02.head()

,nhoodname,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0,0.017241,0.000000,0.000000,0.00,0.017241,0.017241,0.034483,...,0.000000,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.076923,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,Central Bay Street,0.000000,0.0,0.000000,0.014925,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.014925,0.0,0.0,0.014925,0.0,0.014925
3,Church and Wellesley,0.012987,0.0,0.000000,0.000000,0.012987,0.00,0.000000,0.000000,0.000000,...,0.012987,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.025974
4,"Commerce Court, Victoria Hotel",0.040000,0.0,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,...,0.000000,0.0,0.0,0.0,0.020000,0.0,0.0,0.010000,0.0,0.000000


In [24]:
clusters = 2
tvcategoriesv03 = tvcategoriesv02.drop('nhoodname', 1)
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(tvcategoriesv03)
kmeans.labels_

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1], dtype=int32)

In [25]:
tvcategoriesv02.insert(0, 'cluster', kmeans.labels_)

In [26]:
nhooddfv05 = pd.merge(nhooddfv04, tvcategoriesv02, left_on='neighbourhood', right_on='nhoodname', how='inner')
nhooddfv05 = nhooddfv05[['postalcode', 'borough', 'neighbourhood', 'lat', 'long', 'cluster']]
nhooddfv05.head()

,postalcode,borough,neighbourhood,lat,long,cluster
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0


In [27]:
clustersv01 = tvcategoriesv02.groupby('cluster').mean().reset_index()
clustersv01.head()

,cluster,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0.014628,0.002137,0.008699,0.001689,0.002886,0.006271,0.005805,0.004735,0.016204,...,0.000866,0.000000,0.000000,0.001333,0.010161,0.000667,0.000667,0.006122,0.000694,0.011449
1,1,0.002381,0.003704,0.002273,0.000000,0.006878,0.004545,0.000000,0.001587,0.029299,...,0.000000,0.002703,0.002381,0.000000,0.012165,0.002941,0.009307,0.003860,0.000000,0.010452


In [28]:
for cluster in clustersv01['cluster']:
    print('cluster:', cluster)
    temp = clustersv01[clustersv01['cluster'] == cluster].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    #temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(10))
    print('\n')

cluster: 0
                 venue      freq
0          Coffee Shop  0.114229
1                 Café  0.056347
2           Restaurant  0.030735
3                Hotel  0.027086
4                 Park  0.020620
5       Sandwich Place  0.020545
6  Japanese Restaurant  0.019613
7       Clothing Store  0.019411
8   Italian Restaurant  0.019153
9   Seafood Restaurant  0.018869


cluster: 1
                venue      freq
0                Café  0.067370
1         Coffee Shop  0.048878
2  Italian Restaurant  0.040450
3          Restaurant  0.037723
4         Pizza Place  0.035620
5                 Bar  0.030828
6              Bakery  0.029299
7    Greek Restaurant  0.024023
8      Sandwich Place  0.023832
9    Sushi Restaurant  0.023426




In [29]:
clustersmap = folium.Map(location=[latitude, longitude], zoom_start=12.2)

x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(nhooddfv05['lat'], nhooddfv05['long'], nhooddfv05['neighbourhood'], nhooddfv05['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clustersmap)
       
clustersmap